In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np


from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import time
from pprint import pprint
app = Nominatim(user_agent="tutorial")
import os, sys

In [ ]:
data = pd.read_csv('expanded_data_with_OSM_v3.csv', sep = ',')

In [ ]:
data.head(2)

,Unnamed: 0,id,atm_group,address,address_rus,lat,lon,target,geometry,distance_to_retail,...,states,population,salary,population_density,happy_index,avgC,avgR,avgS,avgA,capital
0,0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150,POINT (15889556.987767845 5932448.267111904),7.538905,...,Sakhalin Oblast,8124053,42000,1.168499,77,0.045208,0.031983,0.038556,0.012995,0
1,1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971,POINT (15891230.008394975 5931854.4756764155),272.351877,...,Sakhalin Oblast,8124053,42000,1.168499,77,0.045208,0.031983,0.038556,0.012995,0


In [ ]:
data_with_points = pd.read_csv('all_object.csv', sep = ',')

In [ ]:
len(data_with_points)

5846502

In [ ]:
data_with_points.head(2)

,bulding_type,geometry,lon,lat
0,retail,POINT (37.53998433511941 55.91601868840449),37.539984,55.916019
1,retail,POINT (37.431339550000004 55.84574095000001),37.431340,55.845741


In [ ]:
df_regions = pd.read_csv('regions_data.csv', sep = ';')

In [ ]:
df_regions

,Unnamed: 0,population,salary,population_density,happy_index
0,Central Federal District,39250960,60000,60.367053,84
1,Siberian Federal District,17003927,42845,3.778890,80
2,Volga Federal District,29070827,44500,28.034260,80
3,Far Eastern Federal District,8124053,42000,1.168499,77
4,Northwestern Federal District,13941959,52223,8.264487,76
5,Southern Federal District,16482488,35245,36.805974,86
6,Ural Federal District,12329500,47572,6.780050,80
7,North Caucasian Federal District,9967301,24399,58.480166,83


In [ ]:
#тестовая координата для сбора фичей
lat = 46.937353
lon = 142.753348

### 1. Добавляю объекты OSM

In [ ]:
point = pd.DataFrame([[lon], [lat], ]).T.rename(columns={0:'lon',1:'lat'})

In [ ]:
point['atm_group'] =32

In [ ]:
point['geometry']='POINT ('+str(lon)+' '+str(lat)+')'

In [ ]:
point = gpd.GeoDataFrame(point, geometry=gpd.points_from_xy(point.lon, point.lat))
point.geometry=point.geometry.set_crs('EPSG:4326').to_crs('EPSG:3857')

In [ ]:
data_with_points= gpd.GeoDataFrame(data_with_points[['bulding_type', 'lon', 'lat']], geometry=gpd.points_from_xy(data_with_points.lon, data_with_points.lat))
data_with_points.geometry=data_with_points.geometry.set_crs('EPSG:4326').to_crs('EPSG:3857')

In [ ]:
objects = data_with_points.bulding_type.unique()

for i in objects:
  col_name = 'distance_to_'+i
  point[col_name]= point.\
  sjoin_nearest(data_with_points[data_with_points['bulding_type']==i], how = 'inner', max_distance=10000, distance_col='distance')\
  ['distance'].min()

In [ ]:
point.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   lon                              1 non-null      float64 
 1   lat                              1 non-null      float64 
 2   atm_group                        1 non-null      int64   
 3   geometry                         1 non-null      geometry
 4   distance_to_retail               1 non-null      float64 
 5   distance_to_apartments           1 non-null      float64 
 6   distance_to_commercial           1 non-null      float64 
 7   distance_to_office               1 non-null      float64 
 8   distance_to_train_station        1 non-null      float64 
 9   distance_to_residential          1 non-null      float64 
 10  distance_to_house                1 non-null      float64 
 11  distance_to_detached             1 non-null      float64 
 12  dist

In [ ]:
point = point.fillna(10000)

### 2. Добавляю города, регионы, штаты

In [ ]:
def get_address_by_location(latitude, longitude, language="en"):
    """This function returns an address as raw from a location
    will repeat until success"""
    # build coordinates string to pass to reverse() function
    coordinates = f"{latitude}, {longitude}"
    # sleep for a second to respect Usage Policy
    time.sleep(1)
    try:
        return app.reverse(coordinates, language=language).raw
    except:
        return get_address_by_location(latitude, longitude)

In [ ]:
d = list(zip(point['lat'], point['lon']))

id = []
id_r = []

cities, regions, states = [], [], []
for i, item in enumerate(d):
    address = get_address_by_location(*item)
    pprint(address)
    try:
        cities.append(address['address']['city'])
    except KeyError:
        try:
            cities.append(address['address']['town'])
        except KeyError:
            try:
                cities.append(address['address']['village'])
            except:
                try:
                    cities.append(address['address']['hamlet'])
                except:
                    try:
                        cities.append(address['address']['municipality'])
                    except:
                        cities.append(address['address']['county'])
                        id.append(i)

    try:
        regions.append(address['address']['region'])
    except:
        regions.append(np.nan)
        id_r.append(i)

    states.append(address['address']['state'])
#     pprint(address)
#     print(*item)

point['city'], point['regions'], point['states']  = cities, regions, states

{'address': {'ISO3166-2-lvl4': 'RU-SAK',
             'city': 'Yuzhno-Sakhalinsk',
             'country': 'Russia',
             'country_code': 'ru',
             'county': 'городской округ Южно-Сахалинск',
             'house_number': '259',
             'postcode': '693013',
             'quarter': 'XIV микрорайон',
             'region': 'Far Eastern Federal District',
             'road': 'Комсомольская улица',
             'state': 'Sakhalin Oblast'},
 'addresstype': 'building',
 'boundingbox': ['46.9372511', '46.9374161', '142.7529169', '142.7536106'],
 'class': 'building',
 'display_name': '259, Комсомольская улица, XIV микрорайон, Yuzhno-Sakhalinsk, '
                 'городской округ Южно-Сахалинск, Sakhalin Oblast, Far Eastern '
                 'Federal District, 693013, Russia',
 'importance': 9.99999999995449e-06,
 'lat': '46.937332350000005',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'http://osm.org/copyright',
 'lon': '142.75326566319933',

In [ ]:
point

,lon,lat,atm_group,geometry,distance_to_retail,distance_to_apartments,distance_to_commercial,distance_to_office,distance_to_train_station,distance_to_residential,...,distance_to_bus_stop,distance_to_railway_station,distance_to_tram_stop,distance_to_railway_halt,distance_to_airport,distance_to_parking,distance_to_parking_underground,city,regions,states
0,142.753348,46.937353,32,POINT (15891230.008 5931854.476),272.351877,9.606115,1329.531977,2570.340984,4464.14332,2182.765224,...,129.723638,3143.092355,10000.0,2995.399455,10000.0,2767.793411,2124.799118,Yuzhno-Sakhalinsk,Far Eastern Federal District,Sakhalin Oblast


### 3. Добавляю популяцию, плотность населения, индекс счастья, зарплату

In [ ]:
point= point.merge(df_regions, how = 'inner', left_on = 'regions', right_on = 'Unnamed: 0').drop('Unnamed: 0', axis =1)

In [ ]:
X_test = (
point.drop(['lat', 'lon','geometry'], axis = 1)
.merge(data[['regions','avgR']].groupby('regions', as_index=False).max('avgR'), how = 'inner', on = 'regions')
.merge(data[['states','avgS']].groupby('states', as_index=False).max('avgS'), how = 'inner', on = 'states')
.merge(data[['cities','avgC']].groupby('cities', as_index=False).max('avgC'), how = 'inner', left_on = 'city',right_on = 'cities')
.merge(data[['atm_group','avgA']].groupby('atm_group', as_index=False).max('avgA'), how = 'inner', on = 'atm_group')
).drop(['city'], axis = 1)

<ipython-input-107-285b30966457>:2: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  point.drop(['lat', 'lon','geometry'], axis = 1)
<ipython-input-108-285b30966457>:2: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  point.drop(['lat', 'lon','geometry'], axis = 1)


In [ ]:
X_test.columns

Index(['atm_group', 'distance_to_retail', 'distance_to_apartments',
       'distance_to_commercial', 'distance_to_office',
       'distance_to_train_station', 'distance_to_residential',
       'distance_to_house', 'distance_to_detached', 'distance_to_supermarket',
       'distance_to_cinema', 'distance_to_pharmacy', 'distance_to_cafe',
       'distance_to_bank', 'distance_to_sber_bank', 'distance_to_restaurant',
       'distance_to_sber_atm', 'distance_to_atm', 'distance_to_fast_food',
       'distance_to_vtb_bank', 'distance_to_convenience',
       'distance_to_clothes', 'distance_to_mobile_phone_shop',
       'distance_to_shoe_shop', 'distance_to_vending_any',
       'distance_to_alfa_bank', 'distance_to_vtb_atm', 'distance_to_ros_bank',
       'distance_to_alfa_atm', 'distance_to_ros_atm',
       'distance_to_vending_parking', 'distance_to_bus_stop',
       'distance_to_railway_station', 'distance_to_tram_stop',
       'distance_to_railway_halt', 'distance_to_airport',
       'dista

In [ ]:
X_test['capital'] = np.where((X_test['cities'] == 'Moscow') | (X_test['cities'] == 'Saint Petersburg'), 1, 0)
X_test['atm_group']=X_test['atm_group'].astype('int64')

In [ ]:
len(X_test.columns)

50

In [ ]:
X_test.to_csv('test_test.csv', index = False)

In [ ]:
X_test = X_test[['atm_group', 'distance_to_retail', 'distance_to_apartments',
       'distance_to_commercial', 'distance_to_office',
       'distance_to_train_station', 'distance_to_residential',
       'distance_to_house', 'distance_to_detached', 'distance_to_supermarket',
       'distance_to_cinema', 'distance_to_pharmacy', 'distance_to_cafe',
       'distance_to_bank', 'distance_to_sber_bank', 'distance_to_restaurant',
       'distance_to_sber_atm', 'distance_to_atm', 'distance_to_fast_food',
       'distance_to_vtb_bank', 'distance_to_convenience',
       'distance_to_clothes', 'distance_to_mobile_phone_shop',
       'distance_to_shoe_shop', 'distance_to_vending_any',
       'distance_to_alfa_bank', 'distance_to_vtb_atm', 'distance_to_ros_bank',
       'distance_to_alfa_atm', 'distance_to_ros_atm',
       'distance_to_vending_parking', 'distance_to_bus_stop',
       'distance_to_railway_station', 'distance_to_tram_stop',
       'distance_to_railway_halt', 'distance_to_airport',
       'distance_to_parking', 'distance_to_parking_underground', 'cities',
       'regions', 'states', 'population', 'salary', 'population_density',
       'happy_index', 'avgC', 'avgR', 'avgS', 'avgA', 'capital']]

### 4. Проверка модели

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.3 MB/s eta 0:00:00


In [134]:
from catboost import CatBoostRegressor
model = CatBoostRegressor() # parameters not required.
model.load_model('catboost_model',format='cbm')

In [135]:
model.predict(X_test)

array([0.02242874])